In [ ]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import psycopg2
import json
import random  # Importer le module random

# Configuration du consommateur Kafka
conf = {
    'bootstrap.servers': 'kafka:9092',  # Adresse du broker Kafka
    'group.id': 'my_group',              # ID du groupe de consommateurs
    'auto.offset.reset': 'earliest',       # Commencer à lire les messages les plus récents
}

# Création d'une instance de consommateur
print("Création du consommateur Kafka...")
consumer = Consumer(conf)

# Inscription au topic
topic = 'Mostodon_topic_stream'  # Remplacez par le nom de votre topic
print(f"Inscription au topic: {topic}...")
consumer.subscribe([topic])

# Configuration de la connexion à PostgreSQL
print("Connexion à PostgreSQL...")
conn = psycopg2.connect(
    host="postgres",  # Changez ceci si nécessaire
    database="DB_Mastodon",
    user="fadi",
    password="fadi"
)
cursor = conn.cursor()

# Création de la table si elle n'existe pas déjà
create_table_query = '''
CREATE TABLE IF NOT EXISTS Mostodon_BRONZ (
    id BIGINT PRIMARY KEY,
    username TEXT NOT NULL,
    display_name TEXT NOT NULL,
    content TEXT NOT NULL,
    created_at TIMESTAMP NOT NULL,  
    favourites_count INT NOT NULL,
    reblogs_count INT NOT NULL,
    replies_count INT NOT NULL
);
'''
cursor.execute(create_table_query)
conn.commit()

try:
    while True:
        # Lire les messages du topic
        msg = consumer.poll(1.0)  # Timeout de 1 seconde
        
        if msg is None:
            continue  # Pas de message reçu
        if msg.error():
            # Gestion des erreurs
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Erreur Kafka: {msg.error()}")
                continue
        
        # Traiter le message
        message_value = msg.value().decode('utf-8')

        # Convertir le message JSON en dictionnaire
        try:
            message_dict = json.loads(message_value)
            print(f"Message reçu : {message_dict}")  # Afficher le message reçu
            
            # Vérifier que les champs nécessaires sont présents
            required_fields = ['id', 'username', 'display_name', 'content', 'created_at']
            if not all(field in message_dict for field in required_fields):
                print("Message manquant des champs nécessaires, ignoré.")
                continue

            # Générer des valeurs aléatoires pour les compteurs directement avant l'insertion
            favourites_count = random.randint(1, 100)
            reblogs_count = random.randint(1, 100)
            replies_count = random.randint(1, 100)

            # Insertion du message dans PostgreSQL
            insert_query = '''INSERT INTO Mostodon_BRONZ (id, username, display_name, content, created_at, favourites_count, reblogs_count, replies_count) 
                              VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
                              ON CONFLICT (id) DO NOTHING;'''  # Ignorer les doublons basés sur l'ID
            cursor.execute(insert_query, (
                message_dict['id'],
                message_dict['username'],
                message_dict['display_name'],
                message_dict['content'],
                message_dict['created_at'],  # Insertion de la date de création
                favourites_count,              # Utilisation de la valeur aléatoire
                reblogs_count,                 # Utilisation de la valeur aléatoire
                replies_count                   # Utilisation de la valeur aléatoire
            ))
            conn.commit()  # Valider la transaction

            # Afficher le toot inséré
            print(f"Toot inséré : {message_dict['id']} - {message_dict['username']}")

        except json.JSONDecodeError:
            print("Erreur lors du décodage JSON, message ignoré.")
            continue
        except Exception as e:
            print(f"Erreur lors de l'insertion : {e}")
            continue

except KeyboardInterrupt:
    pass
finally:
    # Compter le nombre total d'entrées dans la table 'Mostodon_BRONZ'
    print("Calcul du nombre total d'entrées dans la table 'Mostodon_BRONZ'...")
    cursor.execute("SELECT COUNT(*) FROM Mostodon_BRONZ;")
    count = cursor.fetchone()[0]  # Récupérer le résultat du COUNT
    print(f"Nombre total d'entrées dans la table 'Mostodon_BRONZ' : {count}")

    # Fermer le consommateur et la connexion à PostgreSQL
    consumer.close()
    cursor.close()
    conn.close()
